In [2]:
pip install pandas matplotlib influxdb-client numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.4/745.4 kB 517.1 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 971.0 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
%matplotlib inline
import pandas as pd
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.query_api import QueryApi
import matplotlib.pyplot as plt
import numpy as np

In [4]:
token = 'MlUmh8wWKDAcBiZbBjvJPguGKxoON1ZRqFjVQ_zh8M3NHLcr53WnxbfXX_ByD0Wm7S8G0kb622gj2EX25MzFCA=='
org = "Portfolio_Opt"
url = "http://localhost:8080"
bucket = "Nifty50"

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

In [5]:
symbols = ['ASIANPAINT.NS', 'BAJAJ-AUTO.NS', 'BAJFINANCE.NS', 'HCLTECH.NS', 'INFY.NS', 'TCS.NS', 'TECHM.NS']

start_date = '2023-01-01'  
end_date = '2024-01-01'
all_data = pd.DataFrame()

In [6]:
for symbol in symbols:
    # Construct the Flux query
    query = f'''
    from(bucket: "{bucket}")
      |> range(start: {start_date}, stop: {end_date})
      |> filter(fn: (r) => r["_measurement"] == "market_data")
      |> filter(fn: (r) => r["_field"] == "close")
      |> filter(fn: (r) => r["symbol"] == "{symbol}")
      |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    '''
    # Execute the query
    result = query_api.query_data_frame(query=query)
    if not result.empty:
        result.set_index('_time', inplace=True)
        result = result[['close']]  # Ensure only close data is used
        result.columns = [symbol]  # Rename column to symbol name

        if all_data.empty:
            all_data = result
        else:
            all_data = all_data.join(result, how='outer')

print(all_data)


NewConnectionError: <urllib3.connection.HTTPConnection object at 0x1319afca0>: Failed to establish a new connection: [Errno 61] Connection refused

In [36]:
returns=[]
for symbol in symbols:
    returns.append((all_data[symbol].iloc[-1]-all_data[symbol][0])/all_data[symbol][0])

print(returns)

[np.float64(0.11654767490155057), np.float64(0.9018872935731027), np.float64(0.11810032424184627), np.float64(0.41045749622967187), np.float64(0.012401590822875657), np.float64(0.1631022885942921), np.float64(0.26067362497678304)]


/var/folders/wr/mslv0h8d73n9kkgc30xk05lm0000gn/T/ipykernel_24433/3194192809.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  returns.append((all_data[symbol].iloc[-1]-all_data[symbol][0])/all_data[symbol][0])


In [37]:
weights=[0.1966477, 0.13984027, 0.21313399, 0.06534991, 0.108151, 0.18365712, 0.09322002]

portfolio_return=np.dot(returns,weights)

print(portfolio_return)

0.25663191948906605


In [38]:
nifty_data=yf.download('^NSEI', start=start_date, end=end_date)
nifty_return=(nifty_data['Close'].iloc[-1]-nifty_data['Close'][0])/nifty_data['Close'][0]
print(nifty_return)

[*********************100%%**********************]  1 of 1 completed

0.1942003590389879



/var/folders/wr/mslv0h8d73n9kkgc30xk05lm0000gn/T/ipykernel_24433/1972782699.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  nifty_return=(nifty_data['Close'].iloc[-1]-nifty_data['Close'][0])/nifty_data['Close'][0]
